# TCGA-4 Blood RNA-seq hgr1 alignments
```
pi:ababaian
files: ~/Crown/data2/tcga_4_blood
start: 2018 09 19
complete : 2018 10 09
```

## Introduction

hgr1 Alignment of the TCGA-LAML, TARGET-AML and TCGA-DLBCL data. Normal controls are absent but from the 3_generic TCGA run it appears that there is stability at ~0.55 within normal samples and therefore deviation from this may measure hypo-macp.


## Materials and Methods

#### TCGA Data Input

Search Term for limiting files
```
cases.primary_site in ["Blood","Bone Marrow","Lymph Nodes"] and files.data_format in ["BAM"] and files.experimental_strategy in ["RNA-Seq"]
```
- Yields 416 files in 378 cases

Note: I don't have authorization for TARGET-AML data-sets, will have to request access to those seperately. Run analysis on TCGA-AML and TCGA-DLBCL only for now.

- 199 files / 199 cases in TCGA-LAML and TCGA-DLBCL

Data processed for input in `TCGA_4_blood.xlsx`


In [1]:
WORKDIR='/home/artem/Crown/data2/tcga_4_blood'

cd $WORKDIR

In [3]:
INPUT1='tcga_4_blood_runs.txt'

cat $INPUT1

TCGA-FA-8693-01A TCGA-DLBC 3ec63efe-97cf-4fa8-aef6-2047e323a0b1
TCGA-FA-A4BB-01A TCGA-DLBC 9093cbe9-1564-46ce-9d0a-834b5ed4705f
TCGA-FA-A4XK-01A TCGA-DLBC 99beedc4-5e62-44c4-a977-b928ff1563d2
TCGA-FA-A6HN-01A TCGA-DLBC 71b006c9-0a12-42e8-8139-2714b2fa4c00
TCGA-FA-A6HO-01A TCGA-DLBC ffe809d5-9571-42a3-9464-e2e77614a250
TCGA-FA-A7DS-01A TCGA-DLBC ee121993-7660-4d51-80a1-fd517d4d2010
TCGA-FA-A7Q1-01A TCGA-DLBC 92db202e-4ebd-4397-9461-c6c98315b415
TCGA-FA-A82F-01A TCGA-DLBC f17fa9f2-7cad-4205-ba5a-590ba6d4f516
TCGA-FA-A86F-01A TCGA-DLBC ceaa1334-faa7-44fd-8341-bcb7e94921c9
TCGA-FF-8041-01A TCGA-DLBC abd1acf7-d2bf-469b-908a-38050299435b
TCGA-FF-8042-01A TCGA-DLBC 12d5e170-4a9d-4981-9da6-c46fdfaec1a2
TCGA-FF-8043-01A TCGA-DLBC a3ca047f-b341-4662-9248-b18ea2b83a9a
TCGA-FF-8046-01A TCGA-DLBC 49049d76-7faf-4aee-a961-95d914c8a75c
TCGA-FF-8047-01A TCGA-DLBC 796ad189-fedd-4ce9-8bd7-57d44851dce6
TCGA-FF-8061-01A TCGA-DLBC 690075e1-1ad4-4ed4-9013-ffbbec977075
TCGA-FF-8062-01A TCGA-DLBC 2f17cacb-d565

#### Scripts
Echo the run scripts for this analysis


In [4]:
cd $WORKDIR
# Echo 

cat hgr1_align_v2.tcga.sh
echo 
echo
cat queenB.sh
echo 
echo
cat droneB.sh
echo 
echo 

#!/bin/bash
# 1kg_align_v2.tcga.sh
# rDNA alignment pipeline
# 180831 build -- TCGA
# AMI: crown-180813 - ami-0031fd61f932bdef9
# EC2: c4.2xlarge (8cpu / 15 gb)
# EC2: c4.xlarge  (4cpu / 8  gb)
# Storage: 200 Gb
#

# Input Requirements --------------------------

# $1 : Library name and file-output name
# $2 : Library population/analysis set
# $3 : Library UUID

# Control Panel -------------------------------
# CPU
	THREADS='3'

# Sequencing Data
	LIBRARY=$1 # Library/ File name

# TCGA FILE UUID
  UUID=$3

 # FastQ File-names
    FQ0="$LIBRARY.tmp.sort.0.fq"
    FQ1="$LIBRARY.tmp.sort.1.fq"
    FQ2="$LIBRARY.tmp.sort.2.fq"
    
# Read Group Data
# Extract from downloaded BAM file / input
	RGPO=$2 # Patient Population

	#RGSM= # Sample. Patient Identifer
	#RGID= # Read Group ID. Accession Number
    
	RGLB=$LIBRARY # Library Name. Accession Number
	RGPL='ILLUMINA'  # Sequencing Platform.
    
	# Extract Sequencing Run Info
	#  RGPU=$(gzip -dc $FQ1 | head -n1 - | cut -f1 -d':' | cut -f2

### LAML debugging

First run had all LAML files fail 


In [ ]:
### Launch AMI Instance: ami-0031fd61f932bdef9
aws s3 cp s3://crownproject/tcga/scripts/hgr1_align_v2.tcga.sh ./

# commented out shutdown of instance at end of run


#screen -L

bash hgr1_align_v2.tcga.sh TCGA-AB-2970-03A TCGA-LAML 5d00c00c-9cd7-4b33-902a-23480e0b95a1

# ...
#
# Error is arising due to multiple readgroups present in each LAML file
#
# @RG	ID::6162NAAXX_3	SM:TCGA-AB-2970-03A-01T-0734-13	PL:ILLUMINA
# @RG	ID::613RTAAXX_3	SM:TCGA-AB-2970-03A-01T-0734-13	PL:ILLUMINA
#
#

RGID=$(~/bin/samtools view -H input.bam | grep '^@RG' | sed "s/.*ID:\([^\t]*\).*/\1/g" | uniq )

echo $RGSM
#TCGA-AB-2970-03A-01T-0734-13
echo $RGID
#:6162NAAXX_3 :613RTAAXX_3


# Can be corrected by changing to...
RGID=$(~/bin/samtools view -H input.bam | grep '^@RG' | sed "s/.*ID:\([^\t]*\).*/\1/g" | head -n1 - )

# CMD from error bowtie run
/usr/bin/bowtie2-align-s --wrapper basic-0 --very-sensitive-local -p 3 \
--rg-id :6162NAAXX_3 :613RTAAXX_3 \
--rg LB:TCGA-AB-2970-03A \
--rg SM:TCGA-AB-2970-03A-01T-0734-13 \
--rg PL:ILLUMINA \
--rg PU:@SOLEXA4_1 \
-x hgr1 -1 TCGA-AB-2970-03A.tmp.sort.1.fq -2 TCGA-AB-2970-03A.tmp.sort.2.fq \


#### LAML Run 2

Set-up parameters for LAML run 2. Deleted the s3 crownproject TCGA-LAML



In [2]:
cat hgr1_align_v2.tcga.sh # with debug on line 70

#!/bin/bash
# 1kg_align_v2.tcga.sh
# rDNA alignment pipeline
# 180831 build -- TCGA
# AMI: crown-180813 - ami-0031fd61f932bdef9
# EC2: c4.2xlarge (8cpu / 15 gb)
# EC2: c4.xlarge  (4cpu / 8  gb)
# Storage: 200 Gb
#

# Input Requirements --------------------------

# $1 : Library name and file-output name
# $2 : Library population/analysis set
# $3 : Library UUID

# Control Panel -------------------------------
# CPU
	THREADS='3'

# Sequencing Data
	LIBRARY=$1 # Library/ File name

# TCGA FILE UUID
  UUID=$3

 # FastQ File-names
    FQ0="$LIBRARY.tmp.sort.0.fq"
    FQ1="$LIBRARY.tmp.sort.1.fq"
    FQ2="$LIBRARY.tmp.sort.2.fq"
    
# Read Group Data
# Extract from downloaded BAM file / input
	RGPO=$2 # Patient Population

	#RGSM= # Sample. Patient Identifer
	#RGID= # Read Group ID. Accession Number
    
	RGLB=$LIBRARY # Library Name. Accession Number
	RGPL='ILLUMINA'  # Sequencing Platform.
    
	# Extract Sequencing Run Info
	#  RGPU=$(gzip -dc $FQ1 | head -n1 - | cut -f1 -d':' | cut -f2

In [4]:
cat tcga_4_blood_runs2.txt

TCGA-AB-2805-03A TCGA-LAML 681cc50b-1644-4f92-a74b-963f82e15834
TCGA-AB-2806-03A TCGA-LAML 4735c487-e74d-4c92-a0c9-2856d611ece8
TCGA-AB-2808-03A TCGA-LAML 91c11b03-6681-4dad-b520-87ee2f4e7d3f
TCGA-AB-2810-03A TCGA-LAML ccb3bdc1-4fda-4447-8a6d-ef56d0407a4b
TCGA-AB-2811-03B TCGA-LAML 775a6469-cde6-4f1b-8536-175431bec2e1
TCGA-AB-2812-03A TCGA-LAML d4d7a8fc-f9a4-4746-a2c7-739e5678583a
TCGA-AB-2813-03A TCGA-LAML 1397f168-e023-42db-b9ee-daa265ee9de8
TCGA-AB-2814-03A TCGA-LAML 2b211dde-de44-4c57-8552-0bad4b90e8db
TCGA-AB-2815-03A TCGA-LAML d6768bd7-6ddf-4707-8b18-f853e08eb496
TCGA-AB-2817-03A TCGA-LAML 896b1ee3-adaf-42d4-ab3a-282714e821ad
TCGA-AB-2818-03A TCGA-LAML 2e42b3cf-f814-42f8-a142-1222fb644a13
TCGA-AB-2819-03A TCGA-LAML b730a8f7-a33c-42ee-b1f8-f19aac843dda
TCGA-AB-2820-03A TCGA-LAML 4a35febf-ef32-4688-8ea8-07ce82d85340
TCGA-AB-2821-03A TCGA-LAML 5ee2d52d-60f2-4848-9ce1-7e1d459220a0
TCGA-AB-2822-03A TCGA-LAML feb1991f-56a0-4b93-8bfc-dfe1b3070501
TCGA-AB-2823-03A TCGA-LAML 375bef78-a998

## Results



### hgr1 Alignments

In [6]:
# Local Folder Operations -----------------------------
# also re-ran with update
# LOCAL:
cd $WORKDIR

INPUT2='tcga_4_blood_runs2.txt'

aws s3 cp queenB.sh s3://crownproject/tcga/scripts/
aws s3 cp droneB.sh s3://crownproject/tcga/scripts/
aws s3 cp hgr1_align_v2.tcga.sh s3://crownproject/tcga/scripts/
#aws s3 cp $INPUT1 s3://crownproject/tcga/scripts/
aws s3 cp $INPUT2 s3://crownproject/tcga/scripts/

upload: ./queenB.sh to s3://crownproject/tcga/scripts/queenB.sh
upload: ./droneB.sh to s3://crownproject/tcga/scripts/droneB.sh
upload: ./hgr1_align_v2.tcga.sh to s3://crownproject/tcga/scripts/hgr1_align_v2.tcga.sh
upload: ./tcga_4_blood_runs2.txt to s3://crownproject/tcga/scripts/tcga_4_blood_runs2.txt


In [1]:
# Remote EC2 Instance Operations ----------------------

# Remote:
# Manually open an Amazon Linux 2 AMI
# ami-6cd6f714
# t2.micro
#
# ssh login:
# ssh -i "crown.pem" ec2-user@PUBLICDNS
#

# Commands on EC2 machine to set-up AWS
# enter personal login info:

# REMOTE:
#aws configure
  # AWS Key ID
  # AWS Secret Key ID
  # Region: us-west-2
  
# Copy local run files to S3 and download them on EC2

# REMOTE:
# aws s3 cp --recursive s3://crownproject/tcga/scripts/ ./
#
# mv <KEY>.pem ~/.ssh/
# chmod 400 ~/.ssh/<KEY>.pem

# REMOTE:
# Open logging screen and being launchign EC2 instances
# screen -L
# 
# bash queenB.sh tcga_4_blood_runs.txt

aws s3 cp s3://crownproject/tcga/logs/tcga_4_blood.log ./
cat tcga_4_blood.log

# All TCGA-DLBCL data-sets completed successfully
# All TCGA-LALM data-sets appear to have failed and yielded empty bam file outputs

download: s3://crownproject/tcga/logs/tcga_4_blood.log to ./tcga_4_blood.log





























































































































































































































































































































































































































































































































































































































































































































































































































































































































































From TCGA-AB-2865-03A-1.screenlog
```
Successfully downloaded: 1
[bam_sort_core] merging from 48 files and 3 in-memory blocks...
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 143645134 reads
Warning: Output file ':700GEAAXX_6' was specified without -S.  This will not work in future Bowtie 2 versions.  Please use -S instead.
71822567 reads; of these:
  71822567 (100.00%) were paired; of these:
    69670175 (97.00%) aligned concordantly 0 times
    2147937 (2.99%) aligned concordantly exactly 1 time
    4455 (0.01%) aligned concordantly >1 times
    ----
    69670175 pairs aligned concordantly 0 times; of these:
      666695 (0.96%) aligned discordantly 1 time
    ----
    69003480 pairs aligned 0 times concordantly or discordantly; of these:
      138006960 mates make up the pairs; of these:
        137714662 (99.79%) aligned 0 times
        277152 (0.20%) aligned exactly 1 time
        15146 (0.01%) aligned >1 times
4.13% overall alignment rate
```

But from the initial (non hgr1 flagstat)
```
0 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 mapped (N/A : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
```

Which means there is likely a pipeline issue with naming/name extraction in the LAML data-sets. Will have to be repeated in entirety.


In [2]:
# TCGA-LAML Pilot experiment with updated script

aws s3 cp s3://crownproject/tcga/logs/tcga_4_pilot2.log ./
cat tcga_4_pilot2.log

echo "--------------------------------"

aws s3 cp s3://crownproject/tcga/logs/tcga_4_blood_run2.log ./
cat tcga_4_blood_run2.log


download: s3://crownproject/tcga/logs/tcga_4_pilot2.log to ./tcga_4_pilot2.log




























kec2-user@ip-172-31-22-32:~\[ec2-user@ip-172-31-22-32 ~]$ --------------------------------
download: s3://crownproject/tcga/logs/tcga_4_blood_run2.log to ./tcga_4_blood_run2.log











































































































































































































































































































































































































































































































































































































































































































































## Discussion

These runs appear to have completed succesfully. : )

# GVCF Analysis of DLBCL + LAML
``` 
Date: 2018-11-20
Launched AMI: crown-180914 (ami-096bcb9d18c32d4d5)
C4.4xlarge 
```

In [ ]:
# Download TCGA-DLBC Data to Crown Project AMI
aws s3 cp --recursive "s3://crownproject/tcga/TCGA-DLBC" ./
mkdir TCGA-DLBC 
mv TCGA*.bam TCGA-DLBC/
mv TCGA*.bam.bai TCGA-DLBC/
mv TCGA*.flagstat TCGA-DLBC/

# Download TCGA-LAML Data to Crown Project AMI
mkdir TCGA-LAML
# error:
# aws s3 cp --recursive "s3://crownproject/tcga/TCGA-DLBC" ./TCGA-LAML/
aws s3 cp --recursive "s3://crownproject/tcga/TCGA-LAML" ./TCGA-LAML/

ls TCGA-LAML TCGA-DLBC > ../TCGA4.filelist

# Run GVCF Analysis of TCGA-DLBC and TCGA-LAML
bash ~/scripts/ADcalc_2.sh #see below


In [ ]:
#!/bin/bash
# ADcalc.sh
# Allelic Depth Calculator
# for a position

cd ~/tcga/

# Controls -----------------
DEPTH='100000'
BAMLIST='bam.list.tmp'

#regions in hgr1.fa reference genome
REGIONS=('chr13:1003660-1005529' 'chr13:10219-10340' \
	'chr13:1006622-1006779' 'chr13:1007948-1013018')

#corresponding gene names
GENES=('18S' '5S' '5.8S' '28S')

# Iterate through every TCGA Cancer Type
for TYPE in $(echo "TCGA-DLBC" "TCGA-LAML")
do
    echo Analyzing $TYPE...
    cd $TYPE

    ls *.bam > bam.list.tmp
    ls *.bam >> ../$OUTPUT.bamlist
    ls *.bam >> ../tcga.bamlist

    for index in ${!GENES[*]}
    do
      printf "Started processing %s\n" ${GENES[$index]}
      OUTPUT="../$TYPE.${GENES[$index]}.gvcf"

      # Iterate through every bam file in directory
      # look-up position and return VCF
      bcftools mpileup -f ~/resources/hgr1/hgr1.fa \
        --max-depth $DEPTH -A --min-BQ 30 \
        -a FORMAT/DP,AD \
        -r ${REGIONS[$index]} \
        --ignore-RG \
        -b $BAMLIST | \
        bcftools annotate -x INFO,FORMAT/PL - | \
        bcftools view -O v - \
        >> $OUTPUT

      RESULTS+=("$OUTPUT")
      printf "Done with %s \n" ${GENES[$index]}
      printf "%s\n" ${REGIONS[$index]}

    done
      
    rm bam.list.tmp
    
    cd ..
done

In [ ]:
## Analyses of output copied to S3
aws s3 cp --recursive 181120_tcga4_gvcf s3://crownproject/tcga/


In [6]:
cd ~/Crown/data2/tcga_analysis
mkdir -p 181120_tcga4_gvcf

aws s3 cp --recursive "s3://crownproject/tcga/181120_tcga4_gvcf/" ./181120_tcga4_gvcf/

# Note LAML VCF didn't work; 49 libraries from DLBL analyzed only
# Manually deleted incorrect LAML files

download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-DLBC.28S.gvcf.bamlist to 181120_tcga4_gvcf/TCGA-DLBC.28S.gvcf.bamlist
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA4.filelist to 181120_tcga4_gvcf/TCGA4.filelist
download: s3://crownproject/tcga/181120_tcga4_gvcf/tcga.bamlist to 181120_tcga4_gvcf/tcga.bamlist
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-DLBC.5S.gvcf to 181120_tcga4_gvcf/TCGA-DLBC.5S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.5S.gvcf to 181120_tcga4_gvcf/TCGA-LAML.5S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.5.8S.gvcf to 181120_tcga4_gvcf/TCGA-LAML.5.8S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-DLBC.5.8S.gvcf to 181120_tcga4_gvcf/TCGA-DLBC.5.8S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-DLBC.18S.gvcf to 181120_tcga4_gvcf/TCGA-DLBC.18S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.18S.gvcf to 181120_tcga4_gvcf/TCGA-LAML.18S.gvcf
download: s3:/

In [ ]:
# Ran on REMOTE:
cd tcga

# Download TCGA-LAML Data to Crown Project AMI
mkdir TCGA-LAML
aws s3 cp --recursive "s3://crownproject/tcga/TCGA-LAML" ./TCGA-LAML

ls TCGA-LAML > ../TCGA-LAML.filelist

# Run GVCF Analysis of TCGA-DLBC and TCGA-LAML
bash ~/scripts/ADcalc_2.sh #see below

# Output files copied to S3

In [9]:
WORKDIR='/home/artem/Desktop/Crown/data2/tcga_analysis/181120_tcga4_gvcf'
cd $WORKDIR

aws s3 cp s3://crownproject/tcga/181120_tcga4_gvcf/tcga-laml.bamlist ./

aws s3 cp s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.18S.gvcf ./
aws s3 cp s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.5S.gvcf ./
aws s3 cp s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.5.8S.gvcf ./
aws s3 cp s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.28S.gvcf ./


download: s3://crownproject/tcga/181120_tcga4_gvcf/tcga-laml.bamlist to ./tcga-laml.bamlist
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.18S.gvcf to ./TCGA-LAML.18S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.5S.gvcf to ./TCGA-LAML.5S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.5.8S.gvcf to ./TCGA-LAML.5.8S.gvcf
download: s3://crownproject/tcga/181120_tcga4_gvcf/TCGA-LAML.28S.gvcf to ./TCGA-LAML.28S.gvcf


In [ ]:
# QED 2018-11-22